In [114]:
import json
from collections import Counter
from itertools import groupby
import pandas as pd
from nltk.corpus import names
from nltk.metrics.distance import edit_distance as editDistance
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re 
import os
import difflib 
import logging
import itertools
import numpy as np
from nltk.util import ngrams 
import difflib
from string import punctuation
from termcolor import colored
from IPython.display import clear_output
%matplotlib inline

In [104]:
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [115]:
with open('txt/e1a.json') as f: 
    rawData = f.read()

df = pd.read_json(rawData)

In [116]:
test1 = df.loc[0]['ocr']

In [117]:
tests = df['ocr']

In [118]:
class Text: 
    def __init__(self, raw_text, label, removeStopwords=True): 
        if type(raw_text) == list: 
            # JSTOR critical works come in lists, where each item represents a page. 
            self.text = ' \n '.join(raw_text)
        else: 
            self.text = raw_text
        self.label = label
        self.preprocess(self.text)
        self.tokens = self.getTokens(removeStopwords)
        self.trigrams = self.ngrams(3)
        
    def preprocess(self, text): 
        """ Heals hyphenated words, and maybe other things. """    
        self.text = re.sub(r'([A-Za-z])- ([a-z])', r'\1\2', self.text)

    def getTokens(self, removeStopwords=True): 
        """ Tokenizes the text, breaking it up into words, removing punctuation. """
        tokenizer = nltk.RegexpTokenizer('[a-zA-Z]\w+\'?\w*') # A custom regex tokenizer. 
        #tokenizer = nltk.RegexpTokenizer('\w+|\$[\d\.]+|\S+') # A custom regex tokenizer. 
        spans = list(tokenizer.span_tokenize(self.text))
        # Take note of how many spans there are in the text
        #print(spans)
        self.length = spans[-1][-1] 
        tokens = tokenizer.tokenize(self.text)
        tokens = [ token.lower() for token in tokens ] # make them lowercase
        stemmer = LancasterStemmer()
        tokens = [ stemmer.stem(token) for token in tokens ]
        if not removeStopwords: 
            self.spans = spans
            return tokens
        tokenSpans = list(zip(tokens, spans)) # zip it up
        stopwords = nltk.corpus.stopwords.words('english') # get stopwords
        tokenSpans = [ token for token in tokenSpans if token[0] not in stopwords ] # remove stopwords from zip
        self.spans = [ x[1] for x in tokenSpans ] # unzip; get spans
        return [ x[0] for x in tokenSpans ] # unzip; get tokens
    
    def ngrams(self, n): 
        """ Returns ngrams for the text."""
        return list(ngrams(self.tokens, n))

class MatcherOld: 
    def __init__(self, textObjA, textObjB, threshold=5, ngramSize=3, removeStopwords=True):
        """
        Takes as input two Text() objects, and matches between them.
        """
        self.threshold = threshold
        self.ngramSize = ngramSize
        
        #self.textA, self.textB = Text(fileA, removeStopwords=removeStopwords), \
        #        Text(fileB, removeStopwords=removeStopwords)
        self.textA = textObjA
        self.textB = textObjB 
        
        self.textAgrams = self.textA.ngrams(ngramSize)
        self.textBgrams = self.textB.ngrams(ngramSize)

        self.locationsA = []
        self.locationsB = []

    def getContext(self, text, start, length, context): 
        match = self.getTokensText(text, start, length)
        before = self.getTokensText(text, start-context, context)
        after = self.getTokensText(text, start+length, context)
        match = colored(match, 'red')
        out = " ".join([before, match, after])
        out = out.replace('\n', ' ') # Replace newlines with spaces. 
        out = re.sub('\s+', ' ', out)
        return out

    def getTokensText(self, text, start, length):  
        """ Looks up the passage in the original text, using its spans. """
        matchTokens = text.tokens[start:start+length]
        spans = text.spans[start:start+length]
        if len(spans) == 0: 
            # Don't try to get text or context beyond the end of a text. 
            passage = ""
        else: 
            passage = text.text[spans[0][0]:spans[-1][-1]]
        return passage 

    def getLocations(self, text, start, length, asPercentages=False): 
        """ Gets the numeric locations of the match. """
        spans = text.spans[start:start+length]
        if asPercentages: 
            locations = (spans[0][0]/text.length, spans[-1][-1]/text.length)
        else: 
            locations = (spans[0][0], spans[-1][-1])
        return locations

    def getMatch(self, match, textA, textB, context): 
        length = match.size + self.ngramSize - 1 # offset according to nGram size 
        wordsA = self.getContext(textA, match.a, length, context)
        wordsB = self.getContext(textB, match.b, length, context)
        spansA = self.getLocations(textA, match.a, length)
        spansB = self.getLocations(textB, match.b, length)
        self.locationsA.append(spansA)
        self.locationsB.append(spansB)
        line1 = ('%s: %s %s' % (colored(textA.label, 'green'), spansA, wordsA) )
        line2 = ('%s: %s %s' % (colored(textB.label, 'green'), spansB, wordsB) )
        return line1 + '\n' + line2

    def match(self): 
        """
        This does the main work of finding matching n-gram sequences between
        the texts.
        """
        sequence = SequenceMatcher(None,self.textAgrams,self.textBgrams)
        matchingBlocks = sequence.get_matching_blocks()

        # Only return the matching sequences that are higher than the 
        # threshold given by the user. 
        highMatchingBlocks = [match for match in matchingBlocks if match.size > self.threshold]
    
        numBlocks = len(highMatchingBlocks)
        self.numMatches = numBlocks
        
        if numBlocks > 0: 
            print('%s total matches found.' % numBlocks, flush=True)

        for num, match in enumerate(highMatchingBlocks): 
            print('match: ', match)
            out = self.getMatch(match, self.textA, self.textB, 5)
            print('\n')
            print('match %s:' % (num+1), flush=True)
            print(out, flush=True)

        return self.numMatches, self.locationsA, self.locationsB



In [120]:
test1Text = Text(test1, 'test1')

In [121]:
mm = Text(open('middlemarch.txt').read(), 'Middlemarch')

In [354]:
class ExtendedMatch(): 
    """ 
    Data structure container for a fancy version of a difflib-style
    Match object. The difflib Match class won't work for extended 
    matches, since it only has the properties `a` (start location in 
    text A), `b` (start location in text B), and size. Since our fancy 
    new matches have different sizes in our different texts, we'll need
    two size attributes. 
    """
    def __init__(self, a, b, sizeA, sizeB): 
        self.a = a
        self.b = b
        self.sizeA = sizeA
        self.sizeB = sizeB
        # Whether this is actually two matches that have been fused into one. 
        self.healed = False 
        # Whether this match has been extended from its original boundaries.
        self.extendedBackwards = 0
        self.extendedForwards = 0
    
    def __repr__(self): 
        out = "a: %s, b: %s, size a: %s, size b: %s" % \
                (self.a, self.b, self.sizeA, self.sizeB)
        if self.extendedBackwards: 
            out += ", extended backwards x%s" % self.extendedBackwards
        if self.extendedForwards: 
            out += ", extended forwards x%s" % self.extendedForwards
        if self.healed: 
            out += ", healed"
        return out

class Matcher(): 
    def __init__(self, textObjA, textObjB, threshold=5, ngramSize=3, removeStopwords=True):
         
        """
        Takes as input two Text() objects, and matches between them.
        """
        self.threshold = threshold
        self.ngramSize = ngramSize
        
        #self.textA, self.textB = Text(fileA, removeStopwords=removeStopwords), \
        #        Text(fileB, removeStopwords=removeStopwords)
        self.textA = textObjA
        self.textB = textObjB 
        
        self.textAgrams = self.textA.ngrams(ngramSize)
        self.textBgrams = self.textB.ngrams(ngramSize)

        self.locationsA = []
        self.locationsB = []
        
        self.initial_matches = self.get_initial_matches()
        self.healed_matches = self.heal_neighboring_matches()
        
        self.extended_matches = self.extend_matches()
    
    def get_initial_matches(self): 
        """
        This does the main work of finding matching n-gram sequences between
        the texts.
        """
        sequence = SequenceMatcher(None,self.textAgrams,self.textBgrams)
        matchingBlocks = sequence.get_matching_blocks()
        
        # Only return the matching sequences that are higher than the 
        # threshold given by the user. 
        highMatchingBlocks = [match for match in matchingBlocks if match.size > self.threshold]
    
        numBlocks = len(highMatchingBlocks)
        self.numMatches = numBlocks
        
        if numBlocks > 0: 
            print('%s total matches found.' % numBlocks, flush=True)
        
        return highMatchingBlocks
    
    def getContext(self, text, start, length, context): 
        match = self.getTokensText(text, start, length)
        before = self.getTokensText(text, start-context, context)
        after = self.getTokensText(text, start+length, context)
        match = colored(match, 'red')
        out = " ".join([before, match, after])
        out = out.replace('\n', ' ') # Replace newlines with spaces. 
        out = re.sub('\s+', ' ', out)
        return out

    def getTokensText(self, text, start, length):  
        """ Looks up the passage in the original text, using its spans. """
        matchTokens = text.tokens[start:start+length]
        spans = text.spans[start:start+length]
        if len(spans) == 0: 
            # Don't try to get text or context beyond the end of a text. 
            passage = ""
        else: 
            passage = text.text[spans[0][0]:spans[-1][-1]]
        return passage 

    def getLocations(self, text, start, length, asPercentages=False): 
        """ Gets the numeric locations of the match. """
        spans = text.spans[start:start+length]
        if asPercentages: 
            locations = (spans[0][0]/text.length, spans[-1][-1]/text.length)
        else: 
            locations = (spans[0][0], spans[-1][-1])
        return locations

    def getMatch(self, match, context=5):
        textA, textB = self.textA, self.textB
        lengthA = match.sizeA + self.ngramSize -1 # offset according to nGram size
        lengthB = match.sizeB + self.ngramSize -1 # offset according to nGram size
        wordsA = self.getContext(textA, match.a, lengthA, context)
        wordsB = self.getContext(textB, match.b, lengthB, context)
        spansA = self.getLocations(textA, match.a, lengthA)
        spansB = self.getLocations(textB, match.b, lengthB)
        self.locationsA.append(spansA)
        self.locationsB.append(spansB)
        line1 = ('%s: %s %s' % (colored(textA.label, 'green'), spansA, wordsA) )
        line2 = ('%s: %s %s' % (colored(textB.label, 'green'), spansB, wordsB) )
        out = line1 + '\n' + line2
        return out

    def heal_neighboring_matches(self, minDistance=8): 
        healedMatches = []
        ignoreNext = False
        matches = self.initial_matches.copy()
        for i, match in enumerate(self.initial_matches): 
            if i+1 > len(self.initial_matches)-1: 
                break
            nextMatch = self.initial_matches[i+1]
            if ignoreNext: 
                ignoreNext = False
                continue
            else: 
                if ( nextMatch.a - (match.a + match.size) ) < minDistance: 
                    print('Potential healing candidate found: ')
                    print(match, nextMatch)
                    sizeA = (nextMatch.a + nextMatch.size) - match.a
                    sizeB = (nextMatch.b + nextMatch.size) - match.b
                    healed = ExtendedMatch(match.a, match.b, sizeA, sizeB)
                    healed.healed = True
                    healedMatches.append(healed)
                    ignoreNext = True
                else: 
                    sizeA, sizeB = match.size, match.size
                    match = ExtendedMatch(match.a, match.b, sizeA, sizeB)
                    healedMatches.append(match)
        return healedMatches
            
    def edit_ratio(self, wordA, wordB): 
        """ Computes the number of edits required to transform one
        (stemmed already, probably) word into another word, and
        adjusts for the average number of letters in each. 
        
        Examples: 
        color, colour: 0.1818181818
        theater, theatre: 0.2857
        day, today: 0.5
        foobar, foo56bar: 0.2857
        """ 
        distance = editDistance(wordA, wordB)
        averageLength = (len(wordA) + len(wordB))/2
        return distance/averageLength
    
    def extend_matches(self, cutoff=0.4): 
        extended = False
        for match in self.healed_matches: 
            # Look one word before. 
            wordA = self.textAgrams[(match.a - 1)][0]
            wordB = self.textBgrams[(match.b - 1)][0]
            if self.edit_ratio(wordA, wordB) < cutoff: 
                print('Extending match backwards with words: %s %s' % 
                     (wordA, wordB) )
                match.a -= 1
                match.b -= 1
                match.sizeA += 1
                match.sizeB += 1
                match.extendedBackwards += 1
                extended = True
            # Look one word after. 
            wordA = self.textAgrams[(match.a + match.sizeA + 1)][-1]
            wordB = self.textBgrams[(match.b + match.sizeB + 1)][-1]
            if self.edit_ratio(wordA, wordB) < cutoff: 
                print('Extending match forwards with words: %s %s' % 
                     (wordA, wordB) )
                match.sizeA += 1
                match.sizeB += 1
                match.extendedForwards += 1
                extended = True
        
        if extended: 
            # If we've gone through the whole list and there's nothing
            # left to extend, then stop. Otherwise do this again. 
            self.extend_matches()
            
        return self.healed_matches
    

    def match(self): 
        """ Gets and prints all matches. """
        
        for num, match in enumerate(self.extended_matches): 
            print('match: ', match)
            out = self.getMatch(match)
            print('\n')
            print('match %s:' % (num+1), flush=True)
            print(out, flush=True)

        return self.numMatches, self.locationsA, self.locationsB

In [355]:
matcher = Matcher(mm, test1Text)

20 total matches found.
Potential healing candidate found: 
Match(a=820, b=1007, size=9) Match(a=834, b=1063, size=13)
Potential healing candidate found: 
Match(a=157584, b=4386, size=12) Match(a=157599, b=4401, size=15)
Extending match forwards with words: destiny destiny
Extending match forwards with words: stand stand
Extending match forwards with words: sarcast sarcast
Extending match forwards with words: dram dram
Extending match forwards with words: persona persona
Extending match forwards with words: fold fold
Extending match forwards with words: hand hand


In [356]:
matcher.match()

match:  a: 550, b: 959, size a: 37, size b: 37


match 1:
Middlemarch: (5809, 6218) interest in gimp and artificial protrusions of drapery mind was theoretic, and yearned by its nature after some lofty conception of the world which might frankly include the parish of Tipton and her own rule of conduct there; she was enamoured of intensity and greatness, and rash in embracing whatever seemed to her to have those aspects; likely to seek martyrdom, to make retractations, and then to incur martyrdom after all in a quarter where she had not sought Certainly such elements in the character of a marriageable girl
test1: (10456, 10865) closely, as the first description of Dorothea shows mind was theoretic, and yearned by its nature after some lofty conception of the world which might frankly include the parish of Tipton and her own rule of conduct there; she was enamoured of intensity and greatness, and rash in embracing whatever seemed to her to have those aspects; likely to seek martyrdom, to

(20,
 [(5809, 6218),
  (8751, 9046),
  (57013, 57100),
  (83868, 83999),
  (116900, 117594),
  (192301, 192441),
  (195148, 195661),
  (402604, 402726),
  (411725, 412177),
  (449403, 450049),
  (450145, 450244),
  (1575265, 1575374),
  (1576340, 1576437),
  (1648982, 1649704),
  (1688955, 1689089),
  (1689907, 1690307),
  (1708999, 1709342)],
 [(10456, 10865),
  (10994, 11737),
  (12404, 12491),
  (13706, 13837),
  (29281, 29974),
  (34588, 34728),
  (35337, 35849),
  (37573, 37693),
  (41901, 42351),
  (42536, 43181),
  (43201, 43300),
  (44605, 44714),
  (44724, 44821),
  (45654, 46366),
  (47146, 47280),
  (47285, 47684),
  (48934, 49270)])